In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("spark-nlp") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:2.4.5") \
    .getOrCreate()

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('words')

[nltk_data] Downloading package stopwords to /home/hadoop/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /home/hadoop/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [3]:
# get the list of stopwords from nltk
from nltk.corpus import stopwords

eng_stopwords = stopwords.words('english')
eng_stopwords.append('rt')
eng_stopwords.append('qt')
eng_stopwords.append('&amp')
eng_stopwords.append('amp')
eng_stopwords.append('+')
eng_stopwords.append('w')

In [4]:
from sparknlp.base import Finisher, DocumentAssembler
from sparknlp.annotator import (Tokenizer, Normalizer, 
                                LemmatizerModel, StopWordsCleaner)
from pyspark.ml import Pipeline
from pyspark.sql import SQLContext
sqlContext = SQLContext(spark)

In [5]:
documentAssembler = DocumentAssembler() \
    .setInputCol('text_no_links') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

# note normalizer defaults to changing all words to lowercase.
# Use .setLowercase(False) to maintain input case.
normalizer = Normalizer() \
    .setInputCols(['token']) \
    .setOutputCol('normalized') \
    .setLowercase(True)

# note that lemmatizer needs a dictionary. So I used the pre-trained
# model (note that it defaults to english)
lemmatizer = LemmatizerModel.pretrained() \
    .setInputCols(['normalized']) \
    .setOutputCol('lemma') \

stopwords_cleaner = StopWordsCleaner() \
    .setInputCols(['lemma']) \
    .setOutputCol('clean_lemma') \
    .setCaseSensitive(False) \
    .setStopWords(eng_stopwords)

# finisher converts tokens to human-readable output
finisher = Finisher() \
    .setInputCols(['clean_lemma']) \
    .setCleanAnnotations(False)

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]


In [6]:
pipeline = Pipeline() \
    .setStages([
        documentAssembler,
        tokenizer,
        normalizer,
        lemmatizer,
        stopwords_cleaner,
        finisher
    ])

In [91]:
from pyspark.sql.functions import regexp_replace, monotonically_increasing_id, col, when, arrays_zip, explode
from pyspark.sql import Row
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import CountVectorizer , IDF
from pyspark.mllib.linalg import Vector, Vectors
from pyspark.ml.clustering import LDA, LDAModel

In [8]:
congDF = spark.read.csv("s3://502finalprojbucky/congresstweets/data/June2017.csv/*.part",header=True)

In [9]:
congDF = congDF.drop("_c0")
data = congDF.filter(congDF['text'].isNull()==False)

#data.select('text').show(1,False)

In [10]:
noLinks = data.withColumn('index',monotonically_increasing_id())
noLinks = noLinks.withColumn('text_no_links',regexp_replace('text','http.*($|\s)',''))

#noLinks.show(1,False)

In [11]:
# transform text with the pipeline
congress = pipeline.fit(noLinks).transform(noLinks)
#congress.select('finished_clean_lemma').show(10,False)

In [12]:
data = congress.select('finished_clean_lemma').withColumn('index',monotonically_increasing_id())

In [13]:
#TF
cv = CountVectorizer(inputCol="finished_clean_lemma",outputCol="features",
                     vocabSize=3500,minDF = 8.5)

In [14]:
# Fit TF
cvmodel = cv.fit(data)

In [15]:
# Transform
result_cv = cvmodel.transform(data)

In [16]:
num_topics=3
max_iter=100

In [30]:
lda_model = LDA(k=num_topics,maxIter=max_iter,optimizer='online').fit(result_cv.select("index","features"))

In [31]:
transform = lda_model.transform(result_cv)

In [32]:
def extractTopDist(row):
    return row.topicDistribution.toArray().tolist()
DF = transform.rdd.map(extractTopDist)

In [33]:
DF = spark.createDataFrame(DF,["Topic1","Topic2","Topic3"]).withColumn("index_1",monotonically_increasing_id())

In [34]:
CoTopDF = transform.join(DF,transform.index==DF.index_1,'inner')\
.select(["index","finished_clean_lemma","features","Topic1","Topic2","Topic3"])

In [35]:
CongressTopics = CoTopDF.rdd.map(lambda r: r.asDict())\
       .map(lambda r: Row(MaxTopic=[max([i for i in r.items() if i[0]\
                                        not in ["index","finished_clean_lemma","features"]], 
                                        key=lambda kv: kv[1])[0],
                                   max([i for i in r.items() if i[0]\
                                        not in ["index","finished_clean_lemma","features"]], 
                                      key=lambda kv: kv[1])[1]], **r) )\
       .toDF()

In [36]:
TopicsCongress = CongressTopics.withColumn("tweet_content",col("finished_clean_lemma"))\
.withColumn("Idx",col("index"))\
.drop("finished_clean_lemma").drop("index")\
.drop("Topic1").drop("Topic2").drop("Topic3")

In [134]:
TopicsCongress.show(20,False)

+----------------------------+------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---+
|MaxTopic                    |features                                                                                                                      |tweet_content                                                                                                                                                           |Idx|
+----------------------------+------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---+
|[Topic

In [38]:
cvRDD = result_cv.select("features","finished_clean_lemma").rdd

In [39]:
termIdxRDD = cvRDD.map(lambda r: Row(\
                                     Idx=[int(str(r[0].indices[i])) for i in range(len(r[0].indices))],
                                     Term=[r[1][i] for i in range(len(r[0].indices))]))

In [40]:
PairedRDD = termIdxRDD.toDF().withColumn("tmp",arrays_zip("Idx","Term")).withColumn("IdxPairs",explode("tmp")).select("IdxPairs")

In [41]:
PairedDF = PairedRDD.rdd.map(lambda r: Row(Index=r[0][0],
                              Term = r[0][1])).toDF()

In [42]:
#spark.stop()